# Model 5.1 and 5.2: Bagging & Random Forest

## Import the libraries and dataset

In [2]:
### Importing the libraries
import numpy as np
import pandas as pd
#from sklearnex import patch_sklearn

#patch_sklearn()

#plotting lib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


#Sklearn Lib metrics
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, KFold

# Pipelines : 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import set_config


from sklearn.ensemble import RandomForestClassifier

#Missing values : 
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, IterativeImputer, SimpleImputer
import missingno as msno

#Dummy
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn. preprocessing import StandardScaler


pd.set_option("display.max_columns",None)

In [3]:
train_df = pd.read_csv("Data/train.csv")
test_df = pd.read_csv("Data/test.csv")
train_df.head()

,id,occ_code_level2,age,stock_dividends,mig_chg_msa,tax_filer_stat,det_hh_summ,mig_prev_sunbelt,hisp_origin,education,wage_per_hour,capital_losses,vet_question,own_or_self,country_self,mig_move_reg,high_income,hs_college,class_worker,mig_same,unemp_reason,state_prev_res,ind_code_level2,race,country_mother,capital_gains,sex,ind_code_level1,citizenship,union_member,fam_under_18,marital_stat,region_prev_res,mig_chg_reg,country_father,occ_code_level1,full_or_part_emp,weeks_worked,det_hh_fam_stat,num_emp,vet_benefits
0,1,0,42.0,0.0,NaN,Nonfiler,Householder,NaN,All other,11th grade,0.0,NaN,Not in universe,0,United-States,NaN,0,Not in universe,Not in universe,Not in universe under 1 year old,Not in universe,Not in universe,0,Black,United-States,0.0,Female,Not in universe or children,Native- Born in the United States,Not in universe,Not in universe,NaN,Not in universe,NaN,United-States,Not in universe,Not in labor force,0.0,Householder,0,2
1,2,18,56.0,NaN,NaN,NaN,Householder,NaN,All other,High school graduate,0.0,NaN,Not in universe,2,United-States,NaN,1,Not in universe,Self-employed-incorporated,Not in universe under 1 year old,NaN,Not in universe,32,NaN,United-States,NaN,Male,Wholesale trade,Native- Born in the United States,Not in universe,Not in universe,Married-civilian spouse present,Not in universe,NaN,United-States,Sales,Full-time schedules,NaN,Householder,1,2
2,3,26,26.0,NaN,NaN,Joint both under 65,Householder,NaN,All other,High school graduate,0.0,NaN,Not in universe,0,Haiti,NaN,0,Not in universe,Private,Not in universe under 1 year old,NaN,Not in universe,41,NaN,Haiti,NaN,NaN,Hospital services,Foreign born- Not a citizen of U S,Not in universe,Not in universe,NaN,Not in universe,NaN,Haiti,Adm support including clerical,Full-time schedules,NaN,Householder,3,2
3,4,0,67.0,NaN,MSA to MSA,Joint one under 65 & one 65+,Householder,No,All other,NaN,0.0,NaN,No,0,United-States,Same county,0,Not in universe,Not in universe,No,NaN,North Carolina,0,Black,United-States,0.0,NaN,Not in universe or children,Native- Born in the United States,Not in universe,Not in universe,NaN,Northeast,Same county,United-States,Not in universe,Children or Armed Forces,0.0,Householder,0,1
4,5,0,NaN,NaN,Nonmover,Nonfiler,Child under 18 never married,Not in universe,All other,Children,0.0,0.0,Not in universe,0,United-States,Nonmover,0,Not in universe,Not in universe,Yes,Not in universe,Not in universe,0,White,United-States,0.0,Female,Not in universe or children,Native- Born in the United States,Not in universe,Mother only present,NaN,Not in universe,Nonmover,United-States,Not in universe,Children or Armed Forces,NaN,Child <18 never marr not in subfamily,0,0


## Cleaning the dataset

In [4]:
#removing id for train_df
train_df.drop("id", axis = 1, inplace = True)

In [5]:
# Transforming object into categories 
# for train
for i in train_df.columns:
    if train_df[i].dtypes == "object":
        train_df[i] = train_df[i].astype("category")
# For test
for i in test_df.columns:
    if test_df[i].dtypes == "object":
        test_df[i] = test_df[i].astype("category")

In [6]:
X= train_df.drop("high_income", axis = 1)
y = train_df["high_income"]

In [7]:
# Grouping our features that are categories in one vector
# Same for numeric

categorical_features = [i for i in X.columns if X[i].dtype.name == "category"]
numerical_features = [i for i in X.columns if X[i].dtype.name != "category"]

## Bagging: First try with 1000 trees

In [9]:

numeric_transformer = Pipeline(
    steps=[("imputer_num", SimpleImputer(strategy="mean")),
           ("scaler", StandardScaler())]
)

categorical_transformer = Pipeline(
    steps=[("imputer_cat", SimpleImputer(strategy="constant", fill_value="Missing")),
           ("encoder" , OneHotEncoder(handle_unknown="ignore", sparse=False))]
)



preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", RandomForestClassifier(random_state=1,n_jobs=-1,max_features=None,n_estimators = 1000))]
)


set_config(display="diagram")
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['occ_code_level2', 'age',
                                                   'stock_dividends',
                                                   'wage_per_hour',
                                                   'capital_losses',
                                                   'own_or_self',
                                                   'ind_code_level2',
                                                   'capital_gains',
                                                   'weeks_worked', 'num_emp',
                                                   'vet_benefits']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer_ca...
                                                   'class_worker', 'mig_same',
                                                   'unemp_reason',
                                                   'state_prev_res', 'race',
                                                   'country_mother', 'sex',
                                                   'ind_code_level1',
                                                   'citizenship',
                                                   'union_member',
                                                   'fam_under_18',
                                                   'marital_stat',
                                                   'region_prev_res',
                                                   'mig_chg_reg',
                                                   'country_father',
                                                   'occ_code_level1',
                                                   'full_or_part_emp',
                                                   'det_hh_fam_stat'])])),
                ('classifier',
                 RandomForestClassifier(max_features=None, n_estimators=1000,
                                        n_jobs=-1, random_state=1))])

In [10]:
clf.fit(X,y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['occ_code_level2', 'age',
                                                   'stock_dividends',
                                                   'wage_per_hour',
                                                   'capital_losses',
                                                   'own_or_self',
                                                   'ind_code_level2',
                                                   'capital_gains',
                                                   'weeks_worked', 'num_emp',
                                                   'vet_benefits']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer_ca...
                                                   'class_worker', 'mig_same',
                                                   'unemp_reason',
                                                   'state_prev_res', 'race',
                                                   'country_mother', 'sex',
                                                   'ind_code_level1',
                                                   'citizenship',
                                                   'union_member',
                                                   'fam_under_18',
                                                   'marital_stat',
                                                   'region_prev_res',
                                                   'mig_chg_reg',
                                                   'country_father',
                                                   'occ_code_level1',
                                                   'full_or_part_emp',
                                                   'det_hh_fam_stat'])])),
                ('classifier',
                 RandomForestClassifier(max_features=None, n_estimators=1000,
                                        n_jobs=-1, random_state=1))])

In [13]:
test_id = test_df[["id"]]
test_df.drop("id",axis = 1, inplace = True)

In [15]:
y_test_pred = clf.predict(test_df)
y_test_pred[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [16]:
test_id["high_income"] = y_test_pred

/var/folders/r3/t8xp3_0d2qg9znqckxzmdc2w0000gp/T/ipykernel_11870/3628057280.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_id["high_income"] = y_test_pred


In [17]:
test_id

,id,high_income
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
6063,6064,0
6064,6065,1
6065,6066,1
6066,6067,0


In [18]:
test_id.to_csv("Predictions/bagging_1000_prediction.csv",index = False, header=True)

## Random Forest: First inference on tuning parameters with automatic GridSearch

In [9]:
### Tuning our tree : 
numeric_transformer = Pipeline(
    steps=[("imputer_num", SimpleImputer(strategy="mean")),
           ("scaler", StandardScaler())]
)

categorical_transformer = Pipeline(
    steps=[("imputer_cat", SimpleImputer(strategy="constant", fill_value="Missing")),
           ("encoder" , OneHotEncoder(handle_unknown="ignore", sparse=False))]
)



preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", RandomForestClassifier(random_state=1,n_jobs=-1,max_features="sqrt",n_estimators = 200))]
)


set_config(display="diagram")
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['occ_code_level2', 'age',
                                                   'stock_dividends',
                                                   'wage_per_hour',
                                                   'capital_losses',
                                                   'own_or_self',
                                                   'ind_code_level2',
                                                   'capital_gains',
                                                   'weeks_worked', 'num_emp',
                                                   'vet_benefits']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer_ca...
                                                   'class_worker', 'mig_same',
                                                   'unemp_reason',
                                                   'state_prev_res', 'race',
                                                   'country_mother', 'sex',
                                                   'ind_code_level1',
                                                   'citizenship',
                                                   'union_member',
                                                   'fam_under_18',
                                                   'marital_stat',
                                                   'region_prev_res',
                                                   'mig_chg_reg',
                                                   'country_father',
                                                   'occ_code_level1',
                                                   'full_or_part_emp',
                                                   'det_hh_fam_stat'])])),
                ('classifier',
                 RandomForestClassifier(n_estimators=200, n_jobs=-1,
                                        random_state=1))])

Below, we just show the number of dimensions of our predictor space, after performing the feature engineering.

In [12]:
preprocessed_data = pd.DataFrame(preprocessor.fit_transform(X))
preprocessed_data
#410 variables ! 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409
0,-0.827102,3.336430e-01,-1.722629e-01,-2.608265e-01,0.000000,-0.344443,-1.026691,-0.194292,-1.531480e+00,-1.008107,0.476955,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.472406,1.223516e+00,-2.391037e-17,-2.608265e-01,0.000000,3.097908,0.712314,0.000000,-3.875967e-16,-0.603743,0.476955,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [14]:
int(np.sqrt(len(preprocessed_data.columns)))

20

**COMMENT**

As seen in class, it is common to use $m \approx \sqrt{p}$. THis is why, we will perform a GridSearch around this number of features.
Thus, we can perform a first automatic GridSearch for Random Forest:

In [20]:
hyper_param = {"classifier__max_features" : np.arange(15,25),
              "classifier__min_samples_leaf": np.arange(1,6,2)}
hyper_param

{'classifier__max_features': array([15, 16, 17, 18, 19, 20, 21, 22, 23, 24]),
 'classifier__min_samples_leaf': array([1, 3, 5])}

In [22]:
rf_grid_search = GridSearchCV(estimator = clf,param_grid=hyper_param, scoring="accuracy",
                           cv = KFold(n_splits=5, shuffle=True, random_state=1), n_jobs=-1, verbose=1)

In [23]:
rf_grid_search.fit(X,y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer_num',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['occ_code_level2',
                                                                          'age',
                                                                          'stock_dividends',
                                                                          'wage_per_hour',
                                                                          'capital_losses',
                                                                          'own_or_self',
                                                                          'ind_code_level2',
                                                                          'capital_gains',
                                                                          'week...
                                                                          'mig_chg_reg',
                                                                          'country_father',
                                                                          'occ_code_level1',
                                                                          'full_or_part_emp',
                                                                          'det_hh_fam_stat'])])),
                                       ('classifier',
                                        RandomForestClassifier(max_features='sqrt',
                                                               n_estimators=200,
                                                               n_jobs=-1,
                                                               random_state=1))]),
             n_jobs=-1,
             param_grid={'classifier__max_features': array([15, 16, 17, 18, 19, 20, 21, 22, 23, 24]),
                         'classifier__min_samples_leaf': array([1, 3, 5])},
             scoring='accuracy', verbose=1)

In [51]:
cv_res_new = pd.DataFrame(rf_grid_search.cv_results_)
cv_res_new

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__max_features,param_classifier__min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,55.925575,1.338859,2.463253,0.184390,15,1,"{'classifier__max_features': 15, 'classifier__...",0.860007,0.860374,0.856332,0.858255,0.857339,0.858461,0.001542,15
1,49.666907,4.867348,2.394748,0.419958,15,3,"{'classifier__max_features': 15, 'classifier__...",0.858726,0.859000,0.854317,0.858438,0.856332,0.857363,0.001791,23
2,38.090263,1.854731,2.082220,0.406755,15,5,"{'classifier__max_features': 15, 'classifier__...",0.856162,0.857535,0.851937,0.857339,0.854958,0.855586,0.002045,30
3,54.741662,1.804889,2.814968,0.529073,16,1,"{'classifier__max_features': 16, 'classifier__...",0.861838,0.860831,0.856423,0.857614,0.859262,0.859194,0.001993,9
4,44.907965,0.634381,2.395191,0.317428,16,3,"{'classifier__max_features': 16, 'classifier__...",0.859366,0.859183,0.855416,0.857888,0.855599,0.857491,0.001698,21
5,41.650819,1.374164,2.217968,0.170026,16,5,"{'classifier__max_features': 16, 'classifier__...",0.859275,0.860190,0.854867,0.856057,0.855691,0.857216,0.002111,27
6,55.189792,1.165297,2.843550,0.169019,17,1,"{'classifier__max_features': 17, 'classifier__...",0.861014,0.862754,0.854226,0.859445,0.856881,0.858864,0.003018,10
7,46.112147,0.798703,2.623310,0.244764,17,3,"{'classifier__max_features': 17, 'classifier__...",0.858359,0.861198,0.854317,0.857156,0.856423,0.857491,0.002272,22
8,42.224755,1.153197,2.499745,0.215450,17,5,"{'classifier__max_features': 17, 'classifier__...",0.857993,0.859824,0.852669,0.856607,0.854592,0.856337,0.002509,29
9,64.241735,3.332654,3.511349,0.748768,18,1,"{'classifier__max_features': 18, 'classifier__...",0.863578,0.861014,0.855966,0.860361,0.859537,0.860091,0.002466,3


In [59]:
mean_scores = cv_res.cv_results_["mean_test_score"]
best_index = np.argmax(mean_scores)
se_scores = cv_res.cv_results_["std_test_score"] / np.sqrt(5)


one_stand_error_data_frame = cv_res_new[cv_res_new["mean_test_score"] >= (mean_scores[best_index] - se_scores[best_index])]
one_stand_error_data_frame[one_stand_error_data_frame["mean_test_score"] == one_stand_error_data_frame["mean_test_score"].min()]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__max_features,param_classifier__min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
18,83.787119,9.696895,4.146263,0.6719,21,1,"{'classifier__max_features': 21, 'classifier__...",0.862113,0.862754,0.85679,0.858255,0.858987,0.85978,0.002288,5


**COMMENT**

It seems that the best model according to the 1 standard-error rule has 21 features and min_samples_leaf of 1.

In [60]:
clf.set_params(classifier__max_features= 21, classifier__min_samples_leaf = 1, classifier__n_estimators = 600)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['occ_code_level2', 'age',
                                                   'stock_dividends',
                                                   'wage_per_hour',
                                                   'capital_losses',
                                                   'own_or_self',
                                                   'ind_code_level2',
                                                   'capital_gains',
                                                   'weeks_worked', 'num_emp',
                                                   'vet_benefits']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer_ca...
                                                   'class_worker', 'mig_same',
                                                   'unemp_reason',
                                                   'state_prev_res', 'race',
                                                   'country_mother', 'sex',
                                                   'ind_code_level1',
                                                   'citizenship',
                                                   'union_member',
                                                   'fam_under_18',
                                                   'marital_stat',
                                                   'region_prev_res',
                                                   'mig_chg_reg',
                                                   'country_father',
                                                   'occ_code_level1',
                                                   'full_or_part_emp',
                                                   'det_hh_fam_stat'])])),
                ('classifier',
                 RandomForestClassifier(max_features=21, n_estimators=600,
                                        n_jobs=-1, random_state=1))])

In [61]:
clf.fit(X,y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['occ_code_level2', 'age',
                                                   'stock_dividends',
                                                   'wage_per_hour',
                                                   'capital_losses',
                                                   'own_or_self',
                                                   'ind_code_level2',
                                                   'capital_gains',
                                                   'weeks_worked', 'num_emp',
                                                   'vet_benefits']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer_ca...
                                                   'class_worker', 'mig_same',
                                                   'unemp_reason',
                                                   'state_prev_res', 'race',
                                                   'country_mother', 'sex',
                                                   'ind_code_level1',
                                                   'citizenship',
                                                   'union_member',
                                                   'fam_under_18',
                                                   'marital_stat',
                                                   'region_prev_res',
                                                   'mig_chg_reg',
                                                   'country_father',
                                                   'occ_code_level1',
                                                   'full_or_part_emp',
                                                   'det_hh_fam_stat'])])),
                ('classifier',
                 RandomForestClassifier(max_features=21, n_estimators=600,
                                        n_jobs=-1, random_state=1))])

### Export the predictions to .csv

In [62]:
y_train_pred = clf.predict(X)
y_train_pred

array([0, 1, 0, ..., 0, 0, 0])

In [65]:
new_tune_200_tree = test_df[["id"]]
new_tune_200_tree

,id
0,1
1,2
2,3
3,4
4,5
...,...
6063,6064
6064,6065
6065,6066
6066,6067


In [66]:
y_test_pred = clf.predict(test_df.drop("id",axis = 1))
y_test_pred

array([0, 0, 0, ..., 0, 0, 1])

In [67]:
new_tune_200_tree["high_income"] = y_test_pred
new_tune_200_tree

/var/folders/r3/t8xp3_0d2qg9znqckxzmdc2w0000gp/T/ipykernel_16032/2699741353.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_tune_200_tree["high_income"] = y_test_pred


,id,high_income
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
6063,6064,0
6064,6065,1
6065,6066,0
6066,6067,0


In [68]:
new_tune_200_tree.to_csv("Predictions/rf_24features_200_prediction.csv",index = False, header=True)

## Random Forest: Trial with 2000 trees

The number of trees is not a tuning parameter. We can increase it's number to decrease the bias !

In [14]:
hyper_param = {"classifier__max_features" : [21],
              "classifier__min_samples_leaf": [1],
              "classifier__n_estimators":[2000]}
rf_grid_search = GridSearchCV(estimator = clf,param_grid=hyper_param, scoring="accuracy",
                           cv = KFold(n_splits=5, shuffle=True, random_state=1), n_jobs=-1, verbose=1)

In [15]:
rf_grid_search.fit(X,y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=1, shuffle=True),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer_num',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['occ_code_level2',
                                                                          'age',
                                                                          'stock_dividends',
                                                                          'wage_per_hour',
                                                                          'capital_losses',
                                                                          'own_or_self',
                                                                          'ind_code_level2',
                                                                          'capital_gains',
                                                                          'week...
                                                                          'marital_stat',
                                                                          'region_prev_res',
                                                                          'mig_chg_reg',
                                                                          'country_father',
                                                                          'occ_code_level1',
                                                                          'full_or_part_emp',
                                                                          'det_hh_fam_stat'])])),
                                       ('classifier',
                                        RandomForestClassifier(n_estimators=200,
                                                               n_jobs=-1,
                                                               random_state=1))]),
             n_jobs=-1,
             param_grid={'classifier__max_features': [21],
                         'classifier__min_samples_leaf': [1],
                         'classifier__n_estimators': [2000]},
             scoring='accuracy', verbose=1)

In [16]:
pd.DataFrame(rf_grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__max_features,param_classifier__min_samples_leaf,param_classifier__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,580.496705,216.264528,16.991444,6.597619,21,1,2000,"{'classifier__max_features': 21, 'classifier__...",0.862754,0.862846,0.855508,0.860086,0.858896,0.860018,0.002724,1


We are havin a mean score of 0.860018 which is the best so far ! 

In [9]:
clf.set_params(classifier__max_features= 21, classifier__min_samples_leaf = 1, classifier__n_estimators = 2000)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['occ_code_level2', 'age',
                                                   'stock_dividends',
                                                   'wage_per_hour',
                                                   'capital_losses',
                                                   'own_or_self',
                                                   'ind_code_level2',
                                                   'capital_gains',
                                                   'weeks_worked', 'num_emp',
                                                   'vet_benefits']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer_ca...
                                                   'class_worker', 'mig_same',
                                                   'unemp_reason',
                                                   'state_prev_res', 'race',
                                                   'country_mother', 'sex',
                                                   'ind_code_level1',
                                                   'citizenship',
                                                   'union_member',
                                                   'fam_under_18',
                                                   'marital_stat',
                                                   'region_prev_res',
                                                   'mig_chg_reg',
                                                   'country_father',
                                                   'occ_code_level1',
                                                   'full_or_part_emp',
                                                   'det_hh_fam_stat'])])),
                ('classifier',
                 RandomForestClassifier(max_features=21, n_estimators=2000,
                                        n_jobs=-1, random_state=1))])

In [10]:
clf.fit(X,y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['occ_code_level2', 'age',
                                                   'stock_dividends',
                                                   'wage_per_hour',
                                                   'capital_losses',
                                                   'own_or_self',
                                                   'ind_code_level2',
                                                   'capital_gains',
                                                   'weeks_worked', 'num_emp',
                                                   'vet_benefits']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer_ca...
                                                   'class_worker', 'mig_same',
                                                   'unemp_reason',
                                                   'state_prev_res', 'race',
                                                   'country_mother', 'sex',
                                                   'ind_code_level1',
                                                   'citizenship',
                                                   'union_member',
                                                   'fam_under_18',
                                                   'marital_stat',
                                                   'region_prev_res',
                                                   'mig_chg_reg',
                                                   'country_father',
                                                   'occ_code_level1',
                                                   'full_or_part_emp',
                                                   'det_hh_fam_stat'])])),
                ('classifier',
                 RandomForestClassifier(max_features=21, n_estimators=2000,
                                        n_jobs=-1, random_state=1))])

### Export the predictions to .csv

In [11]:
new_tune_2000_tree = test_df[["id"]]
new_tune_2000_tree

,id
0,1
1,2
2,3
3,4
4,5
...,...
6063,6064
6064,6065
6065,6066
6066,6067


In [12]:
y_test_pred = clf.predict(test_df.drop("id",axis = 1))
y_test_pred

array([0, 0, 0, ..., 0, 0, 1])

In [13]:
new_tune_2000_tree["high_income"] = y_test_pred
new_tune_2000_tree

/var/folders/r3/t8xp3_0d2qg9znqckxzmdc2w0000gp/T/ipykernel_23702/1942914794.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_tune_2000_tree["high_income"] = y_test_pred


,id,high_income
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
6063,6064,0
6064,6065,1
6065,6066,0
6066,6067,0


In [14]:
new_tune_2000_tree.to_csv("Predictions/rf_21features_2000_prediction.csv",index = False, header=True)